In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# from art.attacks.evasion import SimBA, SpatialTransformation, DeepFool, BasicIterativeMethod, FastGradientMethod, ProjectedGradientDescent
# from art.estimators.classification import PyTorchClassifier

import time

In [13]:
# head = {
#             "model" : '',
#             "attack_model": '',
#             'epsilon': '',
#             'Accuracy': '',
#             'Macro Precision': '',
#             'Weighted Precision': '',
#             'Macro Recall': '',
#             'Weighted Recall': '',
#             'Macro F1': '',
#             'Weighted F1': '',
#             # 'Macro AUC': '',
#             # 'Weighted AUC': '',
#             'TPR': '',
#             'FNR': '',
#             'TNR': '',
#             'FPR': '',
#         }
# head = pd.DataFrame([head])
# head.to_csv("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/RSLAD/RSLAD.csv", mode='a', index=False)


In [14]:
# def calculate_performance_metrics(X_test, y_true, model, model_name, attack_name, eps):
#     model.eval()
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     model.to(device)
    
#     all_preds = []
#     all_labels = []
#     probabilities = []

#     num_classes = len(np.unique(y_true))
    
#     X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
#     y_test_tensor = torch.tensor(y_test, dtype=torch.long)
#     test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
#     test_loader = DataLoader(dataset=test_dataset)

#     with torch.no_grad():
        
#         for inputs, labels in test_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             preds = torch.argmax(outputs, dim=1)
#             all_preds.extend(preds.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())
#             probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
#         all_preds = np.array(all_preds)
#         all_labels = np.array(all_labels)
#         probabilities = np.array(probabilities)
        
#         accuracy = accuracy_score(all_labels, all_preds)

#         precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
#         precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    
#         # macro_auc = roc_auc_score(label_binarize(all_labels, classes=range(num_classes)), probabilities[:,1], average='macro')
#         # weighted_auc = roc_auc_score(label_binarize(all_labels, classes=range(num_classes)), probabilities[:,1], average='weighted')

#         cm = confusion_matrix(all_labels, all_preds)

#         def calculate_class_metrics_macro(cm, class_index):
#             TP = cm[class_index, class_index]
#             FP = cm[:, class_index].sum() - TP
#             FN = cm[class_index, :].sum() - TP
#             TN = cm.sum() - (TP + FP + FN)
            
#             TPR = TP / (TP + FN) if (TP + FN) != 0 else 0  
#             TNR = TN / (TN + FP) if (TN + FP) != 0 else 0  
#             FPR = FP / (FP + TN) if (FP + TN) != 0 else 0  
#             FNR = FN / (FN + TP) if (FN + TP) != 0 else 0  
            
#             return TPR, TNR, FPR, FNR
            
#         metrics = np.array([calculate_class_metrics_macro(cm, i) for i in range(num_classes)])
#         TPR_macro, TNR_macro, FPR_macro, FNR_macro = np.mean(metrics, axis=0)

#         print(f"Accuracy: {accuracy}")
        
#         print("\nmacro")
#         print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")
    
#         print("\nweighted")
#         print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
#         print()
        
#         print(f"Mean FNR: {FNR_macro}\nMean TNR: {TNR_macro}\nMean FPR: {FPR_macro}\nMean TPR: {TPR_macro}")

#         new_row = {
#             "model" : model_name,
#             "attack_model" : attack_name,
#             'epsilon': eps,
#             'Accuracy': accuracy,
#             'Macro Precision': precision_macro,
#             'Weighted Precision': precision_weighted,
#             'Macro Recall': recall_macro,
#             'Weighted Recall': recall_weighted,
#             'Macro F1': f1_macro,
#             'Weighted F1': f1_weighted,
#             # 'Macro AUC': macro_auc,
#             # 'Weighted AUC': weighted_auc,
#             'TPR': TPR_macro,
#             'FNR': FNR_macro,
#             'TNR': TNR_macro,
#             'FPR': FPR_macro,
#         }
#         new_row_df = pd.DataFrame([new_row])
#         new_row_df.to_csv("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/RSLAD/RSLAD.csv", mode='a', index=False, header=False)



In [2]:
def calculate_performance_metrics(X_test, y_true, model, model_name, attack_name, eps):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    all_preds = []
    all_labels = []
    probabilities = []

    num_classes = len(np.unique(y_true))
    
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(dataset=test_dataset)

    with torch.no_grad():
        
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        probabilities = np.array(probabilities)

        np.save(f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense_Label/UNSW_Def9/y_pred_{attack_name}{eps}_Def9.npy", all_preds)


In [3]:
x_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
x_train = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_train.npy')
x_val = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_val.npy')
y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')
y_train = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_train.npy')
y_val = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_val.npy')

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [5]:
input_shape = x_train.shape[1]
output_shape = len(np.unique(y_train))

In [6]:
x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)

x_val_tensor = torch.tensor(x_val, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)

val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=100, shuffle=True)

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.autograd import Variable

class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x


In [8]:
# Initialize model, optimizer, and loss function
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DNNModel(input_size=input_shape, output_size=output_shape).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()

# Early stopping variables
min_delta = 0.001
patience = 5
patience_counter = 0
best_loss = float('inf')


In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import numpy as np

def attack_pgd(model,train_batch_data,train_batch_labels,attack_iters=10,step_size=2/255.0,epsilon=8.0/255.0):
    ce_loss = torch.nn.CrossEntropyLoss().cuda()
    train_ifgsm_data = train_batch_data.detach() + torch.zeros_like(train_batch_data).uniform_(-epsilon,epsilon)
    train_ifgsm_data = torch.clamp(train_ifgsm_data,0,1)
    for i in range(attack_iters):
        train_ifgsm_data.requires_grad_()
        logits = model(train_ifgsm_data)
        loss = ce_loss(logits,train_batch_labels.cuda())
        loss.backward()
        train_grad = train_ifgsm_data.grad.detach()
        train_ifgsm_data = train_ifgsm_data + step_size*torch.sign(train_grad)
        train_ifgsm_data = torch.clamp(train_ifgsm_data.detach(),0,1)
        train_ifgsm_pert = train_ifgsm_data - train_batch_data
        train_ifgsm_pert = torch.clamp(train_ifgsm_pert,-epsilon,epsilon)
        train_ifgsm_data = train_batch_data + train_ifgsm_pert
        train_ifgsm_data = train_ifgsm_data.detach()
    return train_ifgsm_data

def rslad_inner_loss(model,
                teacher_logits,
                x_natural,
                y,
                optimizer,
                step_size=0.003,
                epsilon=0.031,
                perturb_steps=10,
                beta=6.0):
    # define KL-loss
    criterion_kl = nn.KLDivLoss(size_average=False,reduce=False)
    model.eval()
    batch_size = len(x_natural)
    # generate adversarial example
    x_adv = x_natural.detach() + 0.001 * torch.randn(x_natural.shape).cuda().detach()

    for _ in range(perturb_steps):
        x_adv.requires_grad_()
        with torch.enable_grad():
            loss_kl = criterion_kl(F.log_softmax(model(x_adv), dim=1),
                                       F.softmax(teacher_logits, dim=1))
            loss_kl = torch.sum(loss_kl)
        grad = torch.autograd.grad(loss_kl, [x_adv])[0]
        x_adv = x_adv.detach() + step_size * torch.sign(grad.detach())
        x_adv = torch.min(torch.max(x_adv, x_natural - epsilon), x_natural + epsilon)
        x_adv = torch.clamp(x_adv, 0.0, 1.0)

    model.train()

    x_adv = Variable(torch.clamp(x_adv, 0.0, 1.0), requires_grad=False)
    # zero gradient
    optimizer.zero_grad()
    logits = model(x_adv)
    return logits
  

In [10]:
import os
import argparse
import torch
# from rslad_loss import *
# from cifar10_models import *
import torchvision
from torchvision import datasets, transforms
import time
# # we fix the random seed to 0, in the same computer, this method can make the results same as before.
# torch.manual_seed(0)
# torch.cuda.manual_seed_all(0)
# torch.backends.cudnn.deterministic = True

# prefix = 'mobilenet_v2-CIFAR10_RSLAD'
epochs = 10
batch_size = 128
epsilon = 8/255.0



student = DNNModel(input_shape,output_shape )
student = torch.nn.DataParallel(student)
student = student.cuda()
student.train()
optimizer = optim.SGD(student.parameters(), lr=0.1, momentum=0.9, weight_decay=2e-4)
def kl_loss(a,b):
    loss = -a*b + torch.log(b+1e-5)*b
    return loss
teacher = DNNModel(input_shape,output_shape )
teacher.load_state_dict(torch.load('/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/dnn_pytorch.pt'))
teacher = torch.nn.DataParallel(teacher)
teacher = teacher.cuda()
teacher.eval()

for epoch in range(0,epochs):
    train_loss = 0.0
    for step,(train_batch_data,train_batch_labels) in enumerate(train_loader):
        student.train()
        train_batch_data = train_batch_data.float().cuda()
        train_batch_labels = train_batch_labels.cuda()
        optimizer.zero_grad()
        with torch.no_grad():
            teacher_logits = teacher(train_batch_data)

        adv_logits = rslad_inner_loss(student,teacher_logits,train_batch_data,train_batch_labels,optimizer,step_size=2/255.0,epsilon=epsilon,perturb_steps=3)
        student.train()
        nat_logits = student(train_batch_data)
        kl_Loss1 = kl_loss(F.log_softmax(adv_logits,dim=1),F.softmax(teacher_logits.detach(),dim=1))
        kl_Loss2 = kl_loss(F.log_softmax(nat_logits,dim=1),F.softmax(teacher_logits.detach(),dim=1))
        kl_Loss1 = torch.mean(kl_Loss1)
        kl_Loss2 = torch.mean(kl_Loss2)
        loss = 5/6.0*kl_Loss1 + 1/6.0*kl_Loss2
        loss.backward()
        optimizer.step()
        if step % 1000 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, step * len(train_batch_data), len(train_loader.dataset),
                100. * step / len(train_loader), loss.item()))
        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader)

    model.eval()
    val_train_loss = 0.0
    correct_predictions = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = student(inputs)
            kl_Loss1 = kl_loss(F.log_softmax(adv_logits,dim=1),F.softmax(teacher_logits.detach(),dim=1))
            kl_Loss2 = kl_loss(F.log_softmax(nat_logits,dim=1),F.softmax(teacher_logits.detach(),dim=1))
            kl_Loss1 = torch.mean(kl_Loss1)
            kl_Loss2 = torch.mean(kl_Loss2)
            loss = 5/6.0*kl_Loss1 + 1/6.0*kl_Loss2
            
            val_train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            correct_predictions += (predicted == labels).sum().item()

    avg_val_loss = val_train_loss / len(val_loader)
    val_accuracy = correct_predictions / len(val_dataset)

    print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
    # print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}")

    
    # if (epoch%2 == 0):
    #     test_accs = []
    #     student.eval()
    #     for step,(test_batch_data,test_batch_labels) in enumerate(test_loader):
    #         test_ifgsm_data = attack_pgd(student,test_batch_data,test_batch_labels,attack_iters=20,step_size=0.003,epsilon=8.0/255.0)
    #         logits = student(test_ifgsm_data)
    #         predictions = np.argmax(logits.cpu().detach().numpy(),axis=1)
    #         predictions = predictions - test_batch_labels.cpu().detach().numpy()
    #         test_accs = test_accs + predictions.tolist()
    #     test_accs = np.array(test_accs)
    #     test_acc = np.sum(test_accs==0)/len(test_accs)
    #     print('robust acc',np.sum(test_accs==0)/len(test_accs))
    #     # torch.save(student.state_dict(),'./models/'+prefix+str(np.sum(test_accs==0)/len(test_accs))+'.pth')


    # Early stopping check using min_delta
    if best_loss - avg_val_loss > min_delta:
        best_loss = avg_val_loss
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print("Early stopping triggered")
        break

    

/tmp/ipykernel_3820/1642180751.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  teacher.load_state_dict(torch.load('/home/jovyan/Sample_Based_Extension/UNSW/transfer_att

Train Epoch: 0 [0/82332 (0%)]	Loss: 0.308274
Epoch 1, Training Loss: 0.0938, Validation Loss: 0.0752, Validation Accuracy: 0.9196
Train Epoch: 1 [0/82332 (0%)]	Loss: 0.052331
Epoch 2, Training Loss: 0.0640, Validation Loss: 0.0672, Validation Accuracy: 0.9337
Train Epoch: 2 [0/82332 (0%)]	Loss: 0.065983
Epoch 3, Training Loss: 0.0597, Validation Loss: 0.0535, Validation Accuracy: 0.9463
Train Epoch: 3 [0/82332 (0%)]	Loss: 0.039201
Epoch 4, Training Loss: 0.0579, Validation Loss: 0.0618, Validation Accuracy: 0.9178
Train Epoch: 4 [0/82332 (0%)]	Loss: 0.059230
Epoch 5, Training Loss: 0.0562, Validation Loss: 0.0378, Validation Accuracy: 0.9442
Train Epoch: 5 [0/82332 (0%)]	Loss: 0.052274
Epoch 6, Training Loss: 0.0556, Validation Loss: 0.0544, Validation Accuracy: 0.9485
Train Epoch: 6 [0/82332 (0%)]	Loss: 0.065371
Epoch 7, Training Loss: 0.0544, Validation Loss: 0.0429, Validation Accuracy: 0.9539
Train Epoch: 7 [0/82332 (0%)]	Loss: 0.058109
Epoch 8, Training Loss: 0.0538, Validation Lo

In [11]:
calculate_performance_metrics(x_test, y_test, student, 'DNN', 'baseline', '0')

In [12]:
epsilon_values = [0.01, 0.1, 0.2, 0.3]

# Iterate over epsilon values
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, student, 'DNN', 'BIM', epsilon)

for epsilon in epsilon_values:
    filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_FGSM_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, student, 'DNN', 'FGSM', epsilon)

for epsilon in epsilon_values:
    filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_PGD_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, student, 'DNN', 'PGD', epsilon)

In [13]:
epsilon_values = [0.01, 0.1, 0.2, 0.3]

print("start DF")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_DF_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, student, 'DNN', 'DF', epsilon)
    
# Iterate over epsilon values
print("start AutoPGD")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_AutoPGD_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, student, 'DNN', 'AutoPGD', epsilon)

print("start ZOO")
# Iterate over epsilon values
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_ZOO_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, student, 'DNN', 'ZOO', epsilon)

start DF
start AutoPGD
start ZOO


In [12]:
epsilon_values = [0.01, 0.1, 0.2, 0.3]

print("start CaFA")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_CaFA_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, student, 'DNN', 'CaFA', epsilon)

print("start SINIFGSM")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_SINIFGSM_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, student, 'DNN', 'SINIFGSM', epsilon)


print("start VNIFGSM")
# Iterate over epsilon values
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_VNIFGSM_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, student, 'DNN', 'VNIFGSM', epsilon)

start SINIFGSM


In [14]:
torch.save(student.state_dict(), "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/RSLAD/RSLAD_10.pt")

In [ ]:
# student.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/RSLAD/RSLAD_10.pt"))

In [23]:
# import time

# epsilon_values = [0.01, 0.1, 0.2, 0.3]

# start_time = time.time()

# for epsilon in epsilon_values:
#     filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
#     x_test_adv = np.load(filename)

#     calculate_performance_metrics(x_test_adv, y_test, student, 'DNN', 'BIM', epsilon)

# end_time = time.time()
# result = end_time - start_time
# print(f"Execution Time: {result:.6f} seconds")

Accuracy: 0.9148028884540804

macro
Precision: 0.8992260777017462
Recall: 0.9071636606647919
F1 Score: 0.9029979694039452

weighted
Precision: 0.9159500163536447
Recall: 0.9148028884540804
F1 Score: 0.9152221280149041

Mean FNR: 0.09283633933520812
Mean TNR: 0.9071636606647919
Mean FPR: 0.09283633933520812
Mean TPR: 0.9071636606647919
Accuracy: 0.8074685665651788

macro
Precision: 0.7784603987031044
Recall: 0.7928447624940838
F1 Score: 0.7844337653952778

weighted
Precision: 0.8142810183320738
Recall: 0.8074685665651788
F1 Score: 0.8098891497351424

Mean FNR: 0.20715523750591625
Mean TNR: 0.7928447624940838
Mean FPR: 0.20715523750591625
Mean TPR: 0.7928447624940838
Accuracy: 0.5802265488589202

macro
Precision: 0.5828379457825433
Recall: 0.5952505884010408
F1 Score: 0.5672077078155607

weighted
Precision: 0.6485152962666527
Recall: 0.5802265488589202
F1 Score: 0.5943235425196713

Mean FNR: 0.4047494115989591
Mean TNR: 0.5952505884010408
Mean FPR: 0.4047494115989591
Mean TPR: 0.59525058